In [1]:
import numpy as np
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
from keras import layers
from IPython.display import clear_output
clear_output()

2023-01-01 19:57:46.895224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-01 19:58:19.572827: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-01 19:58:19.572882: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
path ="training.1600000.processed.noemoticon.csv"
df = read_csv(path,encoding="ISO-8859-1",usecols=[0,5]).rename(columns = {'pointer':'Col1','text':'Col2'})

In [ ]:
mood,text = df.columns
df = df.rename(columns = {mood:'mood',text:"text"} )
print(df.columns)
df.head()

Index(['mood', 'text'], dtype='object')


,mood,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [5]:
df = df.sample(frac=1,random_state=42) # randomize dataset rows

In [6]:
train_ratio = 0.9
lendf = len(df)
index = int(lendf*train_ratio) # get training / testing index separation

Xtrain = df["text"].head(index) 
Ytrain = df["mood"].head(index)

Xtest = df["text"].tail(lendf-index)
Ytest = df["mood"].tail(lendf-index)

Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)
Xtest = np.array(Xtest)
Ytest = np.array(Ytest)

In [7]:
tokens = 10000
sequence_length = 50 # approx nb of words per tweet

vect_layer = keras.layers.TextVectorization(
    max_tokens = tokens,
    output_mode= 'int',
    output_sequence_length=sequence_length
    )
#we vectorize words that are in training dataset
#we dont adapt on test dataset so that we dont leak information

vect_layer.adapt(Xtrain[:100000]) 
# we could adapt on while dataset but words are mostly the same 
clear_output()

2023-01-01 19:52:41.975636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-01 19:52:41.975838: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-01 19:52:41.975893: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-01 19:52:41.975934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-01 19:52:41.975972: W tensorf

In [8]:
Xtrain_vectorized = vect_layer(Xtrain)
Xtest_vectorized = vect_layer(Xtest)

#one-hot vectorization
def get01(i): #
    """i = 0 ou 4
    4==> [1.0] ==> positif
    0==> [0,1] ==> negatif
    
    """
    if i:
        return [1,0]
    return [0,1]

# be sure to have enough RAM to run this line !
Ytrain_vect = np.array([get01(i) for i in Ytrain]) 
Ytest_vect = np.array([get01(i) for i in Ytest])

2023-01-01 19:52:58.708338: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 300353008 exceeds 10% of free system memory.
2023-01-01 19:52:58.708413: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 450529512 exceeds 10% of free system memory.
2023-01-01 19:53:17.624736: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 575999600 exceeds 10% of free system memory.


In [ ]:
print(Xtrain[0])
print(int(Xtrain_vectorized[0][1]))
print(vect_layer.get_vocabulary()[6])

In [ ]:
embedding_dim = 16 #Dimension of the dense embedding
model = keras.Sequential()
model.add(layers.Embedding(tokens+1,embedding_dim))
model.add(layers.Bidirectional(layers.LSTM(8))) # 8-neuron LSTM
model.add(layers.Flatten()) #dimensions purposes
model.add(layers.Dense(sequence_length,activation="relu"))
model.add(layers.Dense(2,activation= "softmax"))
model.summary()

model.compile(loss="binary_crossentropy",
              optimizer='adam',
             metrics = ["accuracy"])

In [ ]:
epochs=10
h=model.fit(Xtrain_vectorized,Ytrain_vect,batch_size=128,epochs=epochs)
writer = tf.summary.create_file_writer('logs_kaggle')
acc = h.history["accuracy"]
for epoch in range(epochs):
    with writer.as_default():
        tf.summary.scalar('accuracy', acc[epoch], step=epoch)


# to see tensorboard
# type tensorboard --logdir logs_kaggle
# and then go to localhost:6006 to see graph


In [ ]:
acc,loss = model.evaluate(Xtest_vectorized,Ytest_vect)

In [ ]:
# saving model and vectorization layer for future use
#see more at nathanferet.fr/projects/tweet_classification

model.save("mymodel")
import pickle
pickle.dump({'config': vect_layer.get_config(),
             'weights': vect_layer.get_weights()}
            , open("tv_layer.pkl", "wb"))